In [ ]:
import sys
import os
# sys.path.append(top) 把392頂層目錄加到最前面 '/home/pan/django_projects/fast_api_392'
top = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.insert(0, top)
sys.path

In [ ]:
from config import maintenance_html
from apps.ips.config import get_freeproxy_delta
import apps.ips.config as ipscfg
#
print(maintenance_html, get_freeproxy_delta, ipscfg.url_free_us)

In [ ]:
import random
random.randint(0, 7)

# get_freeproxy

In [ ]:
# %%writefile tasks.py
# -*- coding: utf-8 -*-
import asyncio
import requests
import aiohttp
from pyquery import PyQuery as pq
import pandas as pd
import os
import sys
from datetime import datetime
import itertools
#
from fastapi import Request
# 為了在jupyter中試，從apps開始import
import apps.ips.config as ips_cfg
from apps.ips.config import url_free, cacert, ips_csv_path, ips_html_path, dtype, dt_format, ipcols
from apps.ips.utils import aio_get, write_file, csv_update
###############################################################################
# 2021/03/24
# To Do: 檢查ip有效性，aiohttp使用proxy
###############################################################################

async def get_freeproxy(t, once=True):
    get_freeproxy_cnt = 0
    while 1:
        T = (ips_cfg.ips_cycle and os.path.isfile(ips_csv_path))*t  # 沒有 csv 或 ips_cycle 就馬上爬
        await asyncio.sleep(T)
        #
        async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(verify_ssl=cacert)) as session:
            status_code, rtext = await aio_get(session, url_free)
            if status_code == 200 and rtext not in ['', None]:
                doc = pq(rtext, parser='html')
                trs = doc.find('table.table').eq(0).find('tr')
                if trs.size():
                    now = datetime.today().strftime(dt_format)
                    elite = []
                    for tr in trs:
                        tds = pq(tr).find('td')
                        level = tds.eq(4).text().strip()
                        https = tds.eq(6).text().strip()
                        if level != 'elite proxy' or https != 'yes':
                            continue
                        tmp = {
                            'ip': tds.eq(0).text().strip(),
                            'port': tds.eq(1).text().strip(),
                            'level': level,
                            'https': https,
                            'now': now,
                        }
                        elite.append(tmp)
                    # 1 儲存每次撈取的原始頁面
                    write_file(ips_html_path, rtext)
                    # 2 重存csv: 讀取csv檔案，與最新爬的比較
                    if os.path.isfile(ips_csv_path):
                        df1 = pd.read_csv(ips_csv_path, dtype=dtype)
                        df2 = pd.DataFrame(elite).astype(dtype)
                        df3 = csv_update(df1, df2)
                    else:
                        df3 = pd.DataFrame(elite).astype(dtype)
                    df3 = df3.sample(frac=1)  # 亂排
                    df3.to_csv(ips_csv_path, index=False)
                    # 3 更新ips_cycle產生器
                    ips_cfg.ips_cycle = itertools.cycle(df3[ipcols].values.tolist())
                    #
                    get_freeproxy_cnt += 1
                    print(f'get_freeproxy 第{get_freeproxy_cnt}次更新成功:{now}')
                else:
                    pass
        if once:
            break

#
if __name__ == '__main__':
    try:
        tmp = 'zmqshell' in str(type(get_ipython()))  # 在jupyter
        print(tmp, '在jupyter')
    except Exception:
        loop = asyncio.get_event_loop()
        task = loop.create_task(get_freeproxy(1))
        loop.run_until_complete(task)
else:
    if 0:
        asyncio.create_task(get_freeproxy(3, False))

In [ ]:
await get_freeproxy(1)

# ips_check

In [1]:
import requests
import aiohttp
import asyncio
from pyquery import PyQuery as pq
import re
from time import time

In [41]:
cacert = True
proxy_checkurls = [
    "http://210.240.175.62/NTIIS/IP_test.asp",
    "https://whatismyipaddress.com/zh-cn/index",
    "https://httpbin.org/ip",
    "https://www.whatismyip.com.tw/tw/",
    "https://www.whatismyip.com/",
    "https://www.rus.net.tw/myip.php",
    #     "https://www.expressvpn.com/what-is-my-ip",
    #         "https://myip.com.tw/",
    "https://www.myip.com/",
]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}

In [42]:
t = 10
isGood = False
ippt = '51.158.165.18:8811'
proxy = f"http://{ippt}"  # aiohttp只支援http的proxy
rtext = ''
stime = time()
try:
    timeout = aiohttp.ClientTimeout(total=t)
    async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=cacert), timeout=timeout) as session:
        async with session.get(proxy_checkurls[6], headers=headers, proxy=proxy) as r:
            status_code = r.status
#             rtext = await r.text(encoding='utf8')
            rtext = await r.text()
except Exception as err:
    print('err=', err)
else:
    if (status_code == 200) and re.search(ippt.split(":")[0], rtext) is not None:
        isGood = True
    else:
        print('status_code=', status_code, rtext)
finally:
    print('isGood=', isGood, time()-stime)

isGood= True 1.5418686866760254


In [ ]:
rtext

# test

In [ ]:
import aiohttp
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36',
}
a = 'https://www.books.com.tw/products/0010810240?sloc=main'
p = 'http://124.244.22.9:8080'
async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(ssl=True)) as session:
    async with session.get(a, headers=headers, proxy=p) as r:
        status_code = r.status
        rtext = await r.text()
        print(status_code)

In [ ]:
[1, 2, 3, 4, 5, 6][3:]

In [52]:
class A:
    def __init__(self, a, b, c):
        print(1)

In [53]:
b=A(1,2,3,4)

TypeError: __init__() takes 4 positional arguments but 5 were given

In [51]:
A.a

[]

In [ ]:
list

In [ ]:
import pandas as pd

In [ ]:
tmp = [{'a': "1", 'b': 2}, {'a': 1, 'b': 3}, {}]
pd.DataFrame(tmp).values.tolist()

In [ ]:
[d for d in tmp if d]

In [ ]:
sum([1, True, None])

In [ ]:
import asyncio
t = []
#
async def A():
    await asyncio.sleep(1)
    t.append(1)
async def B():
    await asyncio.sleep(4)
    t.append(2)

In [ ]:
t

In [ ]:
tasks = [asyncio.create_task(t()) for t in [A, B]]
await asyncio.wait(tasks)

In [ ]:
t

In [ ]:
for a, b in [[1, 2]]:
    print(a, b)

In [ ]:
a = input()

In [ ]:
[int(n) for n in list(a)]

In [ ]:
import random

In [ ]:
random.sample([1, 2, 3], 2)

In [ ]:
a = [{1: 2}, {2: 3}, {}]
random.shuffle(a)
a

In [ ]:
f'{19.12545:.2}'

In [34]:
class Resistor:
    def __init__(self, ohms):
        self.ohms = ohms
        self._ohms = 123.5
        self.voltage = 0.5
        self.current = 0

# Example 8
class BoundedResistance(Resistor):
    def __init__(self, ohms):
        super().__init__(ohms)
        self.a = 1

    @property
    def ohms(self):
        #         print(321)
        return self._ohms

    @ohms.setter
    def ohms(self, ohms):
        if ohms <= 0:
            raise ValueError(f'ohms must be > 0; got {ohms}')
        self._ohms = ohms


# Example 9
try:
    r3 = BoundedResistance(9)
    print(r3.ohms)
#     r3.ohms = 1
except Exception as err:
    #     logging.exception('Expected')
    print(err)
else:
    print(r3.ohms)

123.5
123.5


In [22]:
r3.ohms

321


{123: 2}

In [24]:
r3._ohms

[10, 0.5]

In [23]:
r3.ohms

321


{123: 2}

In [35]:
class A:
    def __init__(self, a, b):
        self.a = a
        self.b = b

In [39]:
d = A(*[3, 444])

In [40]:
d.a, d.b

(3, 444)

In [42]:
sum([False])

0

In [59]:
a=123
print('11: %s' % a)

11: 123


In [57]:
text = 'world'
print('hello %s' % text)

hello world
